In [3]:
import numpy as np
import pandas as pd
import sklearn
df_train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [4]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
df_train.shape[1]

81

In [6]:
sz=df_train.shape[0]
for column in df_train.columns:
    if(df_train[column].isnull().sum()/sz>=0.4):
        df_train.drop(column,axis=1,inplace=True)
        df_test.drop(column,axis=1,inplace=True)
print(df_train.shape[1])

75


In [7]:
df_train.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,...,112,0,0,0,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,6,2008,WD,Normal,147500


In [8]:
numberofcolumnswithmissing=0
for column in df_train.columns:
    if(df_train[column].isnull().sum()>0):
        print(column,df_train[column].isnull().sum())
        numberofcolumnswithmissing+=1
print(numberofcolumnswithmissing)

LotFrontage 259
MasVnrArea 8
BsmtQual 37
BsmtCond 37
BsmtExposure 38
BsmtFinType1 37
BsmtFinType2 38
Electrical 1
GarageType 81
GarageYrBlt 81
GarageFinish 81
GarageQual 81
GarageCond 81
13


In [9]:
df_train['LotFrontage'].var()

589.7491687482653

In [10]:
df_train['LotFrontage'].value_counts()

LotFrontage
60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
33.0       1
182.0      1
38.0       1
137.0      1
46.0       1
Name: count, Length: 110, dtype: int64

In [11]:
df_train['LotFrontage'].fillna(df_train['LotFrontage'].median())
df_test['LotFrontage'].fillna(df_test['LotFrontage'].median())

0        80.0
1        81.0
2        74.0
3        78.0
4        43.0
        ...  
1454     21.0
1455     21.0
1456    160.0
1457     62.0
1458     74.0
Name: LotFrontage, Length: 1459, dtype: float64

In [12]:
df_train['MasVnrArea'].value_counts()

MasVnrArea
0.0      861
180.0      8
108.0      8
72.0       8
16.0       7
        ... 
567.0      1
337.0      1
51.0       1
234.0      1
63.0       1
Name: count, Length: 327, dtype: int64

In [13]:
df_train['MasVnrArea'].fillna(0)
df_test['MasVnrArea'].fillna(0)

0         0.0
1       108.0
2         0.0
3        20.0
4         0.0
        ...  
1454      0.0
1455      0.0
1456      0.0
1457      0.0
1458     94.0
Name: MasVnrArea, Length: 1459, dtype: float64

In [14]:
df_train['BsmtQual'].value_counts()

BsmtQual
TA    649
Gd    618
Ex    121
Fa     35
Name: count, dtype: int64

In [15]:
print(df_train['BsmtQual'].value_counts().index[0])
for column in df_train.columns:
    if(df_train[column].isnull().sum()>0):
        if(not pd.api.types.is_numeric_dtype(df_train[column])):
            df_train[column]=df_train[column].fillna(df_train[column].value_counts().index[0])
            df_test[column]=df_test[column].fillna(df_test[column].value_counts().index[0])
df_train.head()

TA


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [16]:
df_train_copy=df_train
union_columns = df_train.columns.union(df_test.columns)
for column in union_columns:
    if(pd.api.types.is_numeric_dtype(df_train[column])):
        continue
    mapping={
        True:1,
        False:0
    }
    one_hot_encoded=pd.get_dummies(df_train[column])
    one_hot_encoded.columns = [f"{column}_{class_name}" for class_name in one_hot_encoded.columns]
    L=one_hot_encoded.columns
    df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    df_train.drop(column,axis=1,inplace=True)
    one_hot_encoded=pd.get_dummies(df_test[column])
    one_hot_encoded.columns = [f"{column}_{class_name}" for class_name in one_hot_encoded.columns]
    df_test=pd.concat([df_test,one_hot_encoded],axis=1)
    df_test.drop(column,axis=1,inplace=True)
    for new_column in L:
        df_train[new_column]=df_train[new_column].map(mapping)
        if(new_column in df_test.columns):
            df_test[column]=df_test[new_column].map(mapping)
        else:
            df_test[column]=0
    for new_column in one_hot_encoded:
        if(new_column not in df_train.columns):
            df_train[column]=0
        else:
            df_test[new_column]=df_test[new_column].map(mapping)
df_train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave,Utilities_AllPub,Utilities_NoSeWa
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,0,0,1,0,1,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,0,0,1,0,1,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,0,0,1,0,1,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,0,0,1,0,1,1,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,0,0,1,0,1,1,0


In [17]:
for i in df_train.columns:
    if(i not in df_test.columns):
        df_test[i]=0
for i in df_test.columns:
    if(i not in df_train.columns):
        df_train[i]=0

In [18]:
for column in union_columns:
    if(df_train[column].isnull().sum()>0):
        if(df_train[column].var()/df_train[column].mean()<=0.2):
            df_train[column]=df_train[column].fillna(df_train[column].mean())
            df_test[column]=df_test[column].fillna(df_test[column].mean())
        else:
            df_train[column]=df_train[column].fillna(df_train[column].value_counts().index[0])
            df_test[column]=df_test[column].fillna(df_test[column].value_counts().index[0])
df_train.head(200)
for column in union_columns:
    df_test[column]=df_test[column].fillna(df_test[column].value_counts().index[0])

In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
X=df_train.drop(['Id','SalePrice'],axis=1)
Y=df_train['SalePrice']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
X_train = X_train.astype(float)
X_test = X_test.astype(float)
Y_train = Y_train.astype(float)
Y_test = Y_test.astype(float)
modelensemble=RandomForestClassifier(n_estimators=100,max_depth=5,random_state=4)
modelensemble.fit(X_train,Y_train)
Y_pred=modelensemble.predict(X_test)
mse=mean_squared_error(np.log(Y_pred),np.log(Y_test))
mse=np.sqrt(mse)
print('MSE:',mse)
print(Y_test)

MSE: 0.2745017434382625
892     154500.0
1105    325000.0
413     115000.0
522     159000.0
1036    315500.0
          ...   
479      89471.0
1361    260000.0
802     189000.0
651     108000.0
722     124500.0
Name: SalePrice, Length: 292, dtype: float64


In [57]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test, label=Y_test)
param = {
    'max_depth': 5,
    'eta': 0.145,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}
num_round = 100  # number of boosting rounds
xgbmodel = xgb.train(param, dtrain, num_round)
Y_pred=xgbmodel.predict(dtest)
mse=mean_squared_error(np.log(Y_pred),np.log(Y_test))
mse=np.sqrt(mse)
print('MSE:',mse)

MSE: 0.13355523829738508


In [58]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout
def rmsel(Y_target,Y_pred):
    rmse=tf.math.log(Y_target+1)-tf.math.log(Y_pred+1)
    rmse=rmse*rmse
    rmse = tf.reduce_mean(rmse)
    rmse = tf.sqrt(rmse)
    return rmse
model=Sequential([
    Dense(32,activation='tanh',input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(16,activation='relu'),
    Dense(1,activation='relu')
])
model.compile(optimizer='adam',loss=rmsel)
model.fit(X_train,Y_train,batch_size=1,epochs=1,validation_data=(X_test,Y_test))

1168/1168 [==============================] - 4s 3ms/step - loss: 8.4019 - val_loss: 7.1228


In [59]:
df_test['SalePrice']=0
df_test=df_test[df_train.columns]
X_submission=df_test.drop(['Id','SalePrice'],axis=1)
print(X_submission.shape[1])
X_submission=X_submission.astype(float)
Y_submission=modelensemble.predict(X_submission)
df_submission=df_test['Id']
df_submission = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': Y_submission})
df_submission.to_csv('/kaggle/working/RandForSubmission.csv',index=False)

303


In [60]:
X_submission=df_test.drop(['Id','SalePrice'],axis=1)
X_submission=X_submission.astype(float)
Y_submission=model.predict(X_submission)
print(df_test['Id'].shape)
df_submission=df_test['Id']
df_submission = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': Y_submission.flatten()})
df_submission.to_csv('/kaggle/working/NNSubmission.csv',index=False)

46/46 [==============================] - 0s 1ms/step
(1459,)


In [63]:

X_submission=df_test.drop(['Id','SalePrice'],axis=1)
X_submission=xgb.DMatrix(X_submission)
Y_submission=xgbmodel.predict(X_submission)
print(df_test['Id'].shape)
df_submission=df_test['Id']
df_submission = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': Y_submission.flatten()})
df_submission.to_csv('/kaggle/working/XGboost.csv',index=False)

(1459,)
